In [ ]:
import cv2
import torch
import torchvision.transforms as T
from torchvision.models import resnet18, ResNet18_Weights
from ultralytics import YOLO
import numpy as np
from scipy.spatial.distance import cosine
from time import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Đang sử dụng thiết bị:", device)

print("Đang tải model YOLO...")
yolo_model = YOLO("yolov8n.pt").to(device)
print("Tải model YOLO thành công.")

print("Đang tải model ResNet18...")
weights = ResNet18_Weights.IMAGENET1K_V1
resnet = resnet18(weights=weights).to(device)
resnet.fc = torch.nn.Identity()
resnet.eval()
print("Tải model ResNet18 thành công.")

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

class PersonReIDTracker:
    def __init__(self, dist_threshold=0.3, update_alpha=0.9):
        self.known_persons = {}
        self.next_id = 0
        self.dist_threshold = dist_threshold
        self.update_alpha = update_alpha

    def get_feature_vector(self, crop_img):
        with torch.no_grad():
            input_tensor = transform(crop_img).unsqueeze(0).to(device)
            features = resnet(input_tensor)
        return features.cpu().squeeze().numpy()

    def identify(self, current_feat):
        best_match_id = -1
        min_dist = float('inf')

        for pid, feat in self.known_persons.items():
            dist = cosine(current_feat, feat)
            if dist < min_dist:
                min_dist = dist
                best_match_id = pid

        if min_dist < self.dist_threshold:
            self.known_persons[best_match_id] = (self.update_alpha * self.known_persons[best_match_id] +
                                                 (1 - self.update_alpha) * current_feat)

            self.known_persons[best_match_id] /= np.linalg.norm(self.known_persons[best_match_id])
            return best_match_id, False
        else:
            new_id = self.next_id
            self.known_persons[new_id] = current_feat
            self.next_id += 1
            return new_id, True

reid_tracker = PersonReIDTracker(dist_threshold=0.17)
seen_ids = set()

cap = cv2.VideoCapture("6387-191695740.mp4") #Change to your video file path
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter("output_cnn.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

prev_time = 0

print("\nBắt đầu xử lý video...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time_frame = time()

    results = yolo_model(frame, verbose=False)[0]
    annotated_frame = frame.copy()

    for box in results.boxes:
        cls_id = int(box.cls[0])
        if yolo_model.names[cls_id] != 'person':
            continue

        conf = float(box.conf[0])
        if conf < 0.5:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])

        crop = frame[y1:y2, x1:x2]
        if crop.shape[0] == 0 or crop.shape[1] == 0:
            continue
            
        feat_vec = reid_tracker.get_feature_vector(crop)
        person_id, is_new = reid_tracker.identify(feat_vec)
        
        if is_new:
            print(f"Phát hiện người mới, ID: {person_id}")
        seen_ids.add(person_id)

        label = f"ID: {person_id}"
        color = (0, 255, 0) if not is_new else (0, 0, 255)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated_frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    curr_time = time()
    fps_process = 1 / (curr_time - start_time_frame)
    
    text = f"FPS: {fps_process:.2f}"
    cv2.putText(annotated_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    out.write(annotated_frame)
    cv2.imshow("Person Re-identification", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("\nXử lý hoàn tất.")
print("Danh sách các ID đã ghi nhận:", sorted(list(seen_ids)))
print("Video đầu ra đã được lưu tại: output_cnn.mp4")

Đang sử dụng thiết bị: cuda
Đang tải model YOLO...
Tải model YOLO thành công.
Đang tải model ResNet18...
Tải model ResNet18 thành công.

Bắt đầu xử lý video...
Phát hiện người mới, ID: 0
Phát hiện người mới, ID: 1
Phát hiện người mới, ID: 2
Phát hiện người mới, ID: 3
Phát hiện người mới, ID: 4
Phát hiện người mới, ID: 5
Phát hiện người mới, ID: 6
Phát hiện người mới, ID: 7
Phát hiện người mới, ID: 8
Phát hiện người mới, ID: 9
Phát hiện người mới, ID: 10
Phát hiện người mới, ID: 11
Phát hiện người mới, ID: 12
Phát hiện người mới, ID: 13
Phát hiện người mới, ID: 14
Phát hiện người mới, ID: 15
Phát hiện người mới, ID: 16
Phát hiện người mới, ID: 17
Phát hiện người mới, ID: 18
Phát hiện người mới, ID: 19
Phát hiện người mới, ID: 20
Phát hiện người mới, ID: 21
Phát hiện người mới, ID: 22
Phát hiện người mới, ID: 23
Phát hiện người mới, ID: 24
Phát hiện người mới, ID: 25
Phát hiện người mới, ID: 26
Phát hiện người mới, ID: 27
Phát hiện người mới, ID: 28
Phát hiện người mới, ID: 29
Phát hiện 